Example of subtraction of two nightly coadds

Notebook by Michael Wood-Vasey: <wmwv@pitt.edu>  

In [ ]:
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

import lsst.geom

import lsst.afw.image
from lsst.afw.math import Warper, WarperConfig
import lsst.afw.table

from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask

In [ ]:
# Choose backend
afwDisplay.setDefaultBackend("firefly")

In [ ]:
# Load collection
repo = "embargo_new"
# collection = "LSSTComCam/nightlyValidation"
# As of 2024-11-12 there are observations from 2024-11-08, 2024-11-09, 2024-11-10, 2024-11-11
template_day_obs = "20241108"
collection_template = f"LSSTComCam/runs/nightlyValidation/{template_day_obs}/d_2024_11_05/DM-47059"
science_day_obs = "20241113"
# 20241112 was z band
# collection_science = "LSSTComCam/runs/nightlyValidation/20241112/d_2024_11_05/DM-47059"
collection_science = f"LSSTComCam/runs/nightlyValidation/{science_day_obs}/d_2024_11_05/DM-47059"

instrument = "LSSTComCam"

butler_template = Butler(repo, collections=collection_template)
butler_science = Butler(repo, collections=collection_science)

name_skymap = "lsst_cells_v1"
skymap = butler_template.get("skyMap", skymap=name_skymap, collections="skymaps")

### Identify data_ids from campaign
Get all of the images from ComCam that contain the RA, Dec of the host galaxy

In [ ]:
tract, patch = (5063, 24)
band = "r"

template_coadd = butler_template.get(
            "deepCoadd_calexp",
            band=band, patch=patch, tract=tract, skymap=name_skymap
        )

science_coadd = butler_science.get(
            "deepCoadd_calexp",
            band=band, patch=patch, tract=tract, skymap=name_skymap
        )

In [ ]:
science_coadd_src = butler_science.get(
            "deepCoadd_meas",
            band=band, patch=patch, tract=tract, skymap=name_skymap
        )

In [ ]:
template = template_coadd
science = science_coadd
science_src = science_coadd_src

template_data_id = {"collection": collection_template, "visit": template_day_obs, "band": band, "tract": tract, "patch": patch}
science_data_id = {"collection": collection_science, "visit": science_day_obs, "band": band, "tract": tract, "patch": patch}

In [ ]:
afw_display = afwDisplay.Display(frame=1)

In [ ]:
def warp(science, template):
    "Warp input template image to WCS and Bounding Box of the science image."
    warper_config = WarperConfig()
    warper = Warper.fromConfig(warper_config)

    science_wcs = science.getWcs()
    science_bbox = science.getBBox()
    
    warped_template = warper.warpExposure(science_wcs, template, destBBox=science_bbox)
    # Add PSF.  I think doing this directly without warping is wrong.
    # At least the x,y mapping should be updated
    warped_template.setPsf(template.getPsf())
    
    return warped_template


def subtract(science, template, source_catalog, task=None, config=None, already_aligned=False):
    # https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py#L196
    if config is None and task is None:
        config = AlardLuptonSubtractConfig()
    if task is None:
        task = AlardLuptonSubtractTask(config=config)
    # Star Selection is done here:
    #   https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py#L603

    if not already_aligned:
        warped_template = warp(science, template)
    else:
        warped_template = template
    
    subtraction = task.run(warped_template, science, source_catalog)
    
    return subtraction


def detect(science, subtraction):
    # Run detection on subtraction
    detect_and_measure_config = DetectAndMeasureConfig()
    detect_and_measure_task = DetectAndMeasureTask(config=detect_and_measure_config)

    detect_and_measure = detect_and_measure_task.run(science,
                                                     subtraction.matchedTemplate,
                                                     subtraction.difference)

    return detect_and_measure

In [ ]:
subtract_config = AlardLuptonSubtractConfig()
# Use signalToNoise cut but turn everything else off
subtract_config.sourceSelector.doFluxLimit = False
subtract_config.sourceSelector.doFlags = False
subtract_config.sourceSelector.doUnresolved = False
subtract_config.sourceSelector.doSignalToNoise = True
subtract_config.sourceSelector.doIsolated = False
subtract_config.sourceSelector.doRequireFiniteRaDec = False
subtract_config.sourceSelector.doRequirePrimary = False
subtract_config.sourceSelector.doSkySources = False

task = AlardLuptonSubtractTask(config=subtract_config)
source_catalog = science_src

In [ ]:
subtraction = subtract(science, template, source_catalog, task=task)  #, already_aligned=True)

In [ ]:
detection_catalog = detect(science, subtraction)
dia_src = detection_catalog.diaSources.asAstropy()

In [ ]:
afwDisplay.setDefaultBackend("firefly")
afw_display = afwDisplay.Display(frame=1)
afw_display.scale("asinh", -1, 5)
afw_display.setMaskTransparency(90)
afw_display.mtv(subtraction.matchedTemplate)

afw_display = afwDisplay.Display(frame=2)
afw_display.scale("asinh", -1, 5)
afw_display.setMaskTransparency(90)
afw_display.mtv(subtraction.matchedScience)

afw_display = afwDisplay.Display(frame=3)
afw_display.scale("linear", "zscale")
afw_display.setMaskTransparency(90)
afw_display.mtv(subtraction.difference)

In [ ]:
snr_threshold = 7.5
good = ~dia_src["slot_Shape_flag"] & (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & ~dia_src["base_PixelFlags_flag_edge"]
good_dia_src = dia_src[good]
print(f"{len(good_dia_src)} good DIA sources found out of {len(dia_src)} detections.")

In [ ]:
for s in good_dia_src:
    afw_display.dot("o", s["base_SdssCentroid_x"], s["base_SdssCentroid_y"], size=20, ctype="blue")

In [ ]:
good_dia_src